In [ ]:
%pip install -r requirements.txt

In [6]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
from pprint import pprint

In [38]:
import pandas as pd

# Input Files
players_file = "../CSVs/Designation.csv"  # Contains match stats and roles
stats_file = "../CSVs/fantasy_points/Overall_MatchWise_fantasy_points.csv"

# Output File
output_file = "testing_dream_team_details.csv"

# Columns for Output
output_columns = [
    "match_id", "date",
    "player1_id", "player1_name", "player1_team", "player1_fantasy_points",
    "player2_id", "player2_name", "player2_team", "player2_fantasy_points",
    "player3_id", "player3_name", "player3_team", "player3_fantasy_points",
    "player4_id", "player4_name", "player4_team", "player4_fantasy_points",
    "player5_id", "player5_name", "player5_team", "player5_fantasy_points",
    "player6_id", "player6_name", "player6_team", "player6_fantasy_points",
    "player7_id", "player7_name", "player7_team", "player7_fantasy_points",
    "player8_id", "player8_name", "player8_team", "player8_fantasy_points",
    "player9_id", "player9_name", "player9_team", "player9_fantasy_points",
    "player10_id", "player10_name", "player10_team", "player10_fantasy_points",
    "player11_id", "player11_name", "player11_team", "player11_fantasy_points"
]

# Read Data
players_data = pd.read_csv(players_file, dtype={"player_id": str, "player_name": str, "role": str})
stats_data = pd.read_csv(stats_file, low_memory=False)

# Drop players with missing roles
players_data = players_data.dropna(subset=['role'])

# Prepare Output
output_data = []

# Define Dream Team Calculation Logic
def calculate_dream_team(players, match_group):
    try:
        # Step 1: Pick one player from each role
        selected_players = []
        for role in ["Batsman", "Bowler", "Wicket-Keeper", "All-Rounder"]:
            role_players = [p for p in players if p["role"] == role]
            if role_players:
                selected_players.append(max(role_players, key=lambda x: x["fantasy_points"]))

        # Step 2: Select remaining players to make a team of 11
        remaining_players = [p for p in players if p not in selected_players]
        remaining_players.sort(key=lambda x: x["fantasy_points"], reverse=True)

        while len(selected_players) < 11:
            next_player = remaining_players.pop(0)
            selected_players.append(next_player)

        # Step 3: Ensure team diversity
        team_counts = pd.DataFrame(selected_players)["team_name"].value_counts()
        if team_counts.max() == 11:
            # Replace the lowest fantasy point player with the highest fantasy point player from the other team
            other_team = [team for team in match_group["team_name"].unique() if team not in team_counts.index][0]
            lowest_fantasy_player = min(selected_players, key=lambda x: x["fantasy_points"])
            replacement_player = max(
                [p for p in players if p["team_name"] == other_team and p not in selected_players],
                key=lambda x: x["fantasy_points"]
            )
            selected_players.remove(lowest_fantasy_player)
            selected_players.append(replacement_player)

        # Step 4: Assign Captain and Vice-captain
        selected_players.sort(key=lambda x: x["fantasy_points"], reverse=True)
        
        # captain = selected_players[0]
        # vice_captain = selected_players[1]
        # Adjust fantasy points for captain and vice-captain
        # for player in selected_players:
        #     if player == captain:
        #         player["fantasy_points"] *= 2
        #     elif player == vice_captain:
        #         player["fantasy_points"] *= 1.5

        return selected_players, "Optimal"

    except Exception as e:
        print(f"Error in calculating dream team: {e}")
        return None, "Error"

# Process Matches
i = 1
for match_id, match_group in stats_data.groupby("match_id"):
    date = match_group["date"].iloc[0]

    # Combine player data with match stats
    match_players = pd.merge(players_data, match_group, on=["player_id", "player_name"])
    players = match_players.to_dict(orient="records")

    # Calculate Dream Team
    selected_team, status = calculate_dream_team(players, match_group)

    if not selected_team or len(selected_team) != 11:
        with open("matches_where_team_could_not_be_formed.txt", 'a') as file:
            file.write(f"{status}, {match_id}, {date}, {len(selected_team)}, {selected_team}\n")
        i += 1
        # if i == 1000:
        #     break
        continue

    # Prepare Row for Output
    row = [match_id, date]
    for player in selected_team:
        row.extend([player["player_id"], player["player_name"], player["team_name"], player["fantasy_points"]])

    output_data.append(row)

    if i % 100 == 0:
        print(f"Processed {i} matches...")
    # if i == 1000:
    #     break
    i += 1

# Write Output to CSV
output_df = pd.DataFrame(output_data, columns=output_columns)
output_df.to_csv(output_file, index=False)

print(f"Dream Team details saved to {output_file}")


Processed 100 matches...
Processed 200 matches...
Processed 300 matches...
Processed 400 matches...
Processed 500 matches...
Processed 600 matches...
Processed 700 matches...
Processed 800 matches...
Processed 900 matches...
Processed 1000 matches...
Processed 1100 matches...
Processed 1200 matches...
Processed 1300 matches...
Processed 1400 matches...
Processed 1500 matches...
Processed 1600 matches...
Processed 1700 matches...
Processed 1800 matches...
Processed 1900 matches...
Processed 2000 matches...
Processed 2100 matches...
Processed 2200 matches...
Processed 2300 matches...
Processed 2400 matches...
Processed 2500 matches...
Processed 2600 matches...
Processed 2700 matches...
Processed 2800 matches...
Processed 2900 matches...
Processed 3000 matches...
Processed 3100 matches...
Processed 3200 matches...
Processed 3300 matches...
Processed 3400 matches...
Processed 3500 matches...
Processed 3600 matches...
Processed 3700 matches...
Processed 3800 matches...
Processed 3900 matche

In [12]:
# Input: List of 22 players
# players = [{'LBWs/Bowled': 0,
#   'Role': 'Batsman',
#   'balls_bowled': 0,
#   'balls_faced': 26,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 26.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 1,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '0827b38d',
#   'player_name': 'TG Mokoena',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 25,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Wicket-Keeper',
#   'balls_bowled': 0,
#   'balls_faced': 17,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 48.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 4,
#   'no_of_sixes': 1,
#   'out': 'out',
#   'player_id': 'ede05b97',
#   'player_name': 'HG Kuhn',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 32,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 12,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 5,
#   'fantasy_points': 12.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 1,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': 'c3d35165',
#   'player_name': 'JA Morkel',
#   'runouts': 0,
#   'runs_conceded': 11,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 0,
#   'balls_faced': 33,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 88.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 1,
#   'no_of_sixes': 4,
#   'out': 'not out',
#   'player_id': 'dce2019b',
#   'player_name': 'F Behardien',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 65,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 12,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 3,
#   'fantasy_points': 0.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'not out',
#   'player_id': 'aa8d28ae',
#   'player_name': 'D Wiese',
#   'runouts': 0,
#   'runs_conceded': 18,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 11,
#   'fantasy_points': 31.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': 'f834dcfc',
#   'player_name': 'L Ngidi',
#   'runouts': 0,
#   'runs_conceded': 18,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 1},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 9,
#   'fantasy_points': 29.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': '9d50c7e1',
#   'player_name': 'MP Siboto',
#   'runouts': 0,
#   'runs_conceded': 23,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 1},
#  {'LBWs/Bowled': 1,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 5,
#   'fantasy_points': 97.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 1,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': 'ffd3fc37',
#   'player_name': 'S von Berg',
#   'runouts': 0,
#   'runs_conceded': 25,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 3},
#  {'LBWs/Bowled': 0,
#   'Role': 'Wicket-Keeper',
#   'balls_bowled': 0,
#   'balls_faced': 33,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 51.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 4,
#   'no_of_sixes': 1,
#   'out': 'out',
#   'player_id': '710dd98c',
#   'player_name': 'MN van Wyk',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 41,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 12,
#   'balls_faced': 12,
#   'date': '2016-11-20',
#   'dot_balls': 4,
#   'fantasy_points': 32.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': 'afa78c3a',
#   'player_name': 'CS Delport',
#   'runouts': 0,
#   'runs_conceded': 11,
#   'runs_scored': 7,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 1},
#  {'LBWs/Bowled': 0,
#   'Role': 'Batsman',
#   'balls_bowled': 0,
#   'balls_faced': 6,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 12.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 1,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '4aad78d3',
#   'player_name': 'K Zondo',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 4,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 6,
#   'balls_faced': 10,
#   'date': '2016-11-20',
#   'dot_balls': 2,
#   'fantasy_points': 26.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '9784fb60',
#   'player_name': 'K Nipper',
#   'runouts': 0,
#   'runs_conceded': 12,
#   'runs_scored': 5,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 1},
#  {'LBWs/Bowled': 0,
#   'Role': 'Batsman',
#   'balls_bowled': 0,
#   'balls_faced': 10,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 10.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'not out',
#   'player_id': '27ca3a11',
#   'player_name': 'C Chetty',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 10,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 8,
#   'date': '2016-11-20',
#   'dot_balls': 6,
#   'fantasy_points': 15.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '8b248d88',
#   'player_name': 'R Frylinck',
#   'runouts': 1,
#   'runs_conceded': 39,
#   'runs_scored': 6,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 1,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 5,
#   'date': '2016-11-20',
#   'dot_balls': 5,
#   'fantasy_points': 38.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'not out',
#   'player_id': 'acee4cc4',
#   'player_name': 'Imran Tahir',
#   'runouts': 0,
#   'runs_conceded': 38,
#   'runs_scored': 5,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 1},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 7,
#   'fantasy_points': 0.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': '016e5101',
#   'player_name': 'M Shezi',
#   'runouts': 0,
#   'runs_conceded': 37,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Wicket-Keeper',
#   'balls_bowled': 0,
#   'balls_faced': 11,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 27.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 1,
#   'no_of_fours': 2,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '235c2bb6',
#   'player_name': 'H Klaasen',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 15,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 24,
#   'balls_faced': 0,
#   'date': '2016-11-20',
#   'dot_balls': 10,
#   'fantasy_points': 87.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 1,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'DNB',
#   'player_id': '1cb14aa4',
#   'player_name': 'CJ Dala',
#   'runouts': 0,
#   'runs_conceded': 30,
#   'runs_scored': 0,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 3},
#  {'LBWs/Bowled': 0,
#   'Role': 'Batsman',
#   'balls_bowled': 0,
#   'balls_faced': 33,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 65.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 2,
#   'no_of_fours': 3,
#   'no_of_sixes': 1,
#   'out': 'out',
#   'player_id': 'a833f99c',
#   'player_name': 'JD Vandiar',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 40,
#   'stumpings': 0,
#   'team_name': 'Titans',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'Bowler',
#   'balls_bowled': 18,
#   'balls_faced': 5,
#   'date': '2016-11-20',
#   'dot_balls': 6,
#   'fantasy_points': 4.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 0,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '7ec9711d',
#   'player_name': 'P Subrayen',
#   'runouts': 0,
#   'runs_conceded': 26,
#   'runs_scored': 4,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 12,
#   'balls_faced': 8,
#   'date': '2016-11-20',
#   'dot_balls': 2,
#   'fantasy_points': 14.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 1,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': 'd0db2e43',
#   'player_name': 'M Gqadushe',
#   'runouts': 0,
#   'runs_conceded': 17,
#   'runs_scored': 13,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0},
#  {'LBWs/Bowled': 0,
#   'Role': 'All-Rounder',
#   'balls_bowled': 0,
#   'balls_faced': 23,
#   'date': '2016-11-20',
#   'dot_balls': 0,
#   'fantasy_points': 26.0,
#   'gender': 'male',
#   'match_id': '1003857',
#   'match_type': 'T20',
#   'no_of_catches': 0,
#   'no_of_fours': 2,
#   'no_of_sixes': 0,
#   'out': 'out',
#   'player_id': '39f01cdb',
#   'player_name': 'KP Pietersen',
#   'runouts': 0,
#   'runs_conceded': 0,
#   'runs_scored': 24,
#   'stumpings': 0,
#   'team_name': 'Dolphins',
#   'venue': 'Bloemfontein',
#   'wickets': 0}]

def calcuate_dream_team(players):
    
    teams = players['team_name'].unique()
    # Define the problem
    print(teams)
    problem = LpProblem("Dream11_Team_Selection", LpMaximize)

    # Create binary decision variables for each player (1 = selected, 0 = not selected)
    player_vars = {player["player_name"]: LpVariable(
        player["player_name"], cat="Binary") for player in players}

    # Create binary decision variables for captain and vice-captain
    captain_vars = {player["player_name"]: LpVariable(
        f"captain_{player['player_name']}", cat="Binary") for player in players}
    vice_captain_vars = {player["player_name"]: LpVariable(
        f"vice_captain_{player['player_name']}", cat="Binary") for player in players}

    # Objective: Maximize total fantasy points with captain and vice-captain bonuses
    problem += lpSum(
        player["fantasy_points"] * player_vars[player["player_name"]] +
        # Double points for captain
        player["fantasy_points"] * captain_vars[player["player_name"]] +
        # 1.5x points for vice-captain
        (player["fantasy_points"] * 0.5) * vice_captain_vars[player["player_name"]]
        for player in players
    )

    # Constraint: Select exactly 11 players
    problem += lpSum(player_vars[player["player_name"]] for player in players) == 11

    # Constraint: At least one player from each team
    for team in teams:
        problem += lpSum(player_vars[player["player_name"]]
                        for player in players if player["team_name"] == team) >= 1

    # Constraint: Role-based composition (1–8 for each role)
    roles = ["Batsman", "Bowler", "All-rounder", "Wicket-keeper"]
    for role in roles:
        role_players = [player["player_name"]
                        for player in players if player["Role"] == role]
        problem += lpSum(player_vars[player] for player in role_players) >= 1
        problem += lpSum(player_vars[player] for player in role_players) <= 8

    # Constraint: Exactly one captain and one vice-captain must be selected
    problem += lpSum(captain_vars[player["player_name"]] for player in players) == 1
    problem += lpSum(vice_captain_vars[player["player_name"]] for player in players) == 1

    # Constraint: Captain and vice-captain must be selected in the team
    for player in players:
        problem += captain_vars[player["player_name"]] <= player_vars[player["player_name"]]
        problem += vice_captain_vars[player["player_name"]] <= player_vars[player["player_name"]]

    # Constraint: Captain and vice-captain must be different players
    for player in players:
        problem += captain_vars[player["player_name"]] + \
            vice_captain_vars[player["player_name"]] <= 1

    # Solve the problem
    problem.solve()

    # Retrieve selected team
    selected_team = [
        {
            "player_name": player["player_name"],
            "team_name": player["team_name"],
            "role": player["Role"],
            "fantasy_points": player["fantasy_points"],
            "is_captain": captain_vars[player["player_name"]].value() == 1,
            "is_vice_captain": vice_captain_vars[player["player_name"]].value() == 1,
        }
        for player in players if player_vars[player["player_name"]].value() == 1
    ]

    # Calculate adjusted fantasy points for each player
    for player in selected_team:
        if player["is_captain"]:
            player["adjusted_points"] = player["fantasy_points"] * 2
        elif player["is_vice_captain"]:
            player["adjusted_points"] = player["fantasy_points"] * 1.5
        else:
            player["adjusted_points"] = player["fantasy_points"]

    # Sort selected team by adjusted fantasy points in descending order
    selected_team.sort(key=lambda x: x["adjusted_points"], reverse=True)

    # Display the team details

    def show(team):
        print(f"{'Name':<15}{'Team':<10}{'Role':<15}{'Fantasy Points':<15}{
            'Adjusted Points':<20}{'Captain':<10}{'Vice-Captain':<15}")
        print("-" * 100)
        for player in team:
            print(
                f"{player['player_name']:<15}{player['team_name']:<10}{
                    player['role']:<15}{player['fantasy_points']:<15}"
                f"{player['adjusted_points']:<20}"
                f"{'Yes' if player['is_captain'] else 'No':<10}{
                    'Yes' if player['is_vice_captain'] else 'No':<15}"
            )

        # Calculate total adjusted points
        total_points = sum(player["adjusted_points"] for player in team)
        print("-" * 100)
        print(f"{'Total Adjusted Points':<55}{total_points:<15}")


    show(selected_team)

players_file = "../CSVs/Designation.csv"  # Contains designation, player_id, player_name
stats_file = "../CSVs/fantasy_points/Overall_MatchWise_fantasy_points.csv"  # Contains match stats and fantasy points

players_data = pd.read_csv(players_file,dtype={"player_id":str,"player_name":str,"Role":str})
stats_data = pd.read_csv(stats_file)

# Create the Dream Team CSV file
output_file = "dream_team_details.csv".upper()

# Columns for output
output_columns = [
    "match_id", "date",
    "player1_id", "player1_name", "player1_fantasy_points",
    "player2_id", "player2_name", "player2_fantasy_points",
    "player3_id", "player3_name", "player3_fantasy_points",
    "player4_id", "player4_name", "player4_fantasy_points",
    "player5_id", "player5_name", "player5_fantasy_points",
    "player6_id", "player6_name", "player6_fantasy_points",
    "player7_id", "player7_name", "player7_fantasy_points",
    "player8_id", "player8_name", "player8_fantasy_points",
    "player9_id", "player9_name", "player9_fantasy_points",
    "player10_id", "player10_name", "player10_fantasy_points",
    "player11_id", "player11_name", "player11_fantasy_points"
]

# Prepare the output CSV
output_data = []

# Process each match
for match_id, match_group in stats_data.groupby("match_id"):
    date = match_group["date"].iloc[0]
    # Combine player data with stats
    match_players = pd.merge(players_data, match_group, on=["player_id", "player_name"])
    players = match_players.to_dict(orient="records")

    # Solve CSP for the match
    selected_team = calcuate_dream_team(players)

    # Prepare row for the output CSV
    row = [match_id, date]
    for player in selected_team:
        row.extend([player["player_id"], player["player_name"], player["fantasy_points"]])

    output_data.append(row)
    break

# Write to CSV
output_df = pd.DataFrame(output_data, columns=output_columns)
output_df.to_csv(output_file, index=False)

print(f"Dream Team details saved to {output_file}")


C:\Users\vrajp\AppData\Local\Temp\ipykernel_23224\2502200318.py:621: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  stats_data = pd.read_csv(stats_file)


TypeError: list indices must be integers or slices, not str

In [ ]:
i

17844